### Proposition 1: List of Products that costs more than the average

**Functional Specification**
* Use a scalar subquery to calculate the average `UnitPrice` from the `Warehouse.StockItems` table.
* Select the `StockItemID`, `StockItemName`, and `UnitPrice` from the `Warehouse.StockItems` table.
* Filter the results in the `WHERE` clause to only include products whose `UnitPrice` is greater than the calculated average.
* Limit the output to the `TOP 7` results.

In [ ]:
USE WideWorldImporters;
SELECT TOP 7 StockItemID,
       StockItemName,
       UnitPrice
FROM Warehouse.StockItems
WHERE UnitPrice > (
      SELECT AVG(UnitPrice)
      FROM Warehouse.StockItems
);
-- Shows products whose UnitPrice is greater than the average.

### Proposition 2: Most recent 5 orders between 2015–2016

**Functional Specification**
* Create a derived table aliased as 'o'.
* Inside the derived table, select `OrderID`, `CustomerID`, and `OrderDate` from the `Sales.Orders` table.
* Filter the records within the derived table to include only orders placed between January 1, 2015, and December 31, 2016.
* Select the specified columns from the derived table.
* Order the final results by `OrderDate` in descending order to show the newest orders first.
* Limit the output to the `TOP 5` most recent orders.

In [ ]:
USE WideWorldImporters;
SELECT TOP 5 o.OrderID,
       o.CustomerID,
       o.OrderDate
FROM (
      SELECT OrderID, CustomerID, OrderDate
      FROM Sales.Orders
      WHERE OrderDate BETWEEN '2015-01-01' AND '2016-12-31'
     ) AS o
ORDER BY o.OrderDate DESC;
-- Retrieves the newest 5 orders placed in 2015–2016.

### Proposition 3: Products starting with 'A' (first 7 by StockItemID)

**Functional Specification**
* Select `StockItemID`, `StockItemName`, and `SupplierID` from the `Warehouse.StockItems` table (aliased as 's').
* Use a correlated scalar subquery in the `SELECT` list to count the total number of items from the same supplier (`s2.SupplierID = s.SupplierID`) whose names also start with 'A'.
* Filter the main query in the `WHERE` clause to only include products whose `StockItemName` starts with the letter 'A'.
* Order the results by `StockItemID` in ascending order.
* Limit the output to the `TOP 7` results.

In [ ]:
USE WideWorldImporters;
SELECT TOP 7
       s.StockItemID,
       s.StockItemName,
       s.SupplierID,
       (
          SELECT COUNT(*)
          FROM Warehouse.StockItems AS s2
          WHERE s2.SupplierID = s.SupplierID
            AND s2.StockItemName LIKE 'A%'
       ) AS ItemsStartingWithA_BySameSupplier
FROM Warehouse.StockItems AS s
WHERE s.StockItemName LIKE 'A%'
ORDER BY s.StockItemID ASC;
-- Filters products whose names start with 'A'

### Proposition 4: Return all products whose unit price is greater than at least one product’s unit price from Stock Group 3.

**Functional Specification**
* Use a subquery with the `ANY` predicate to generate a list of unit prices.
* Inside the subquery, join `Warehouse.StockItems` and `Warehouse.StockItemStockGroups` to filter for products where the `StockGroupID` is 3.
* The outer query selects `StockItemID`, `StockItemName`, and `UnitPrice` from `Warehouse.StockItems`.
* Filter the results in the `WHERE` clause to include any product whose `UnitPrice` is greater than at least one of the prices from the subquery's list.
* Limit the output to the `TOP 8` results.

In [ ]:
USE WideWorldImporters;
SELECT TOP 8 si.StockItemID AS ProductID,
       si.StockItemName AS ProductName,
       si.UnitPrice
FROM Warehouse.StockItems AS si
WHERE si.UnitPrice > ANY (
    SELECT si2.UnitPrice
    FROM Warehouse.StockItems AS si2
    INNER JOIN Warehouse.StockItemStockGroups AS ssg
        ON si2.StockItemID = ssg.StockItemID
    WHERE ssg.StockGroupID = 3
);

### Proposition 5: Most recent order date for each customer

**Functional Specification**
* Select `CustomerID` and `CustomerName` from the `Sales.Customers` table (aliased as 'C').
* Use a correlated scalar subquery in the `SELECT` list to find the latest order date for each customer.
* The subquery calculates the `MAX(OrderDate)` from the `Sales.Orders` table for the corresponding `CustomerID` from the outer query (`O.CustomerID = C.CustomerID`).
* Limit the output to the `TOP 7` customers.

In [ ]:
USE WideWorldImporters;
SELECT TOP 7 C.CustomerID, C.CustomerName,
       (SELECT MAX(O.OrderDate)
        FROM Sales.Orders AS O
        WHERE O.CustomerID = C.CustomerID) AS LatestOrderDate
FROM Sales.Customers AS C;
/*For each customer, this query finds the most recent (latest) order date 
  by using a subquery that looks up the maximum OrderDate in the Orders table.*/

### Proposition 6: Top 7 selling products by total quantity sold

**Functional Specification**
* Create a derived table aliased as 'dt' to pre-calculate sales totals.
* Inside the derived table, group the `Sales.OrderLines` table by `StockItemID` and use `SUM(Quantity)` to find the total quantity sold for each item.
* Join the derived table to the `Warehouse.StockItems` table on `StockItemID` to retrieve the `StockItemName`.
* Order the final results by the `TotalSold` in descending order to rank products from most to least sold.
* Limit the output to the `TOP 7` selling products.

In [ ]:
USE WideWorldImporters;
SELECT TOP 7
       dt.StockItemID,
       si.StockItemName,
       dt.TotalSold
FROM (
        -- Derived table: compute total sold quantity per product
        SELECT ol.StockItemID,
               SUM(ol.Quantity) AS TotalSold
        FROM Sales.OrderLines AS ol
        GROUP BY ol.StockItemID
     ) AS dt
JOIN Warehouse.StockItems AS si
     ON dt.StockItemID = si.StockItemID
ORDER BY dt.TotalSold DESC;
-- Finds the top 7 most sold products based on total quantity ordered.

### Proposition 7: 5 most recent orders

**Functional Specification**
* Create a Common Table Expression (CTE) named `OrderedCTE`.
* Inside the CTE, select order details from the `Sales.Orders` table.
* Use the `ROW_NUMBER()` window function ordered by `OrderDate` descending to assign a sequential rank to each order, starting with 1 for the most recent.
* Select `OrderID`, `CustomerID`, and `OrderDate` from the CTE.
* Filter the results in the `WHERE` clause to keep only rows where the rank (`RowNum`) is less than or equal to 5.
* Order the final result by `OrderDate` descending.

In [ ]:
USE WideWorldImporters;
-- 5 most recent orders using a Common Table Expression
WITH OrderedCTE AS (
    SELECT 
        OrderID,
        CustomerID,
        OrderDate,
        ROW_NUMBER() OVER (ORDER BY OrderDate DESC) AS RowNum
    FROM Sales.Orders
)
SELECT 
    OrderID,
    CustomerID,
    OrderDate
FROM OrderedCTE
WHERE RowNum <= 5
ORDER BY OrderDate DESC;
-- Returns the latest 5 orders from the system.

### Proposition 8: Total order amount per customer (top 5)

**Functional Specification**
* Declare an integer variable `@TopN` and set its value to 5.
* Call the table-valued function `Sales.fn_TopCustomerTotals`, passing the `@TopN` variable as a parameter. This function is assumed to return the top N customers based on their total spending.
* Join the results of the function (aliased as 't') with the `Sales.Customers` table on `CustomerID` to retrieve customer names.
* Order the final result by `TotalAmount` in descending order to show the highest spenders first.

In [ ]:
USE WideWorldImporters;
-- 🔹 Change this value to return different counts (e.g., 5, 10, 20)
DECLARE @TopN INT = 5;

-- 🔹 Call the function and join to get customer names
SELECT 
    c.CustomerID,
    c.CustomerName,
    t.TotalAmount
FROM Sales.fn_TopCustomerTotals(@TopN) AS t
JOIN Sales.Customers AS c
     ON c.CustomerID = t.CustomerID
ORDER BY t.TotalAmount DESC;
-- Calculates which customers spent the most money in total.

### Proposition 9: Customers spending more than $50,000

**Functional Specification**
* Create a CTE named `CustomerTotals`.
* Inside the CTE, join `Sales.Orders` and `Sales.OrderLines`.
* Group the results by `CustomerID` to calculate the `SUM` of `Quantity * UnitPrice` (as `TotalSpent`) and the `MAX(OrderDate)` (as `MostRecentOrder`) for each customer.
* Join the `CustomerTotals` CTE with the `Sales.Customers` table to get customer names.
* Filter the results in the `WHERE` clause to only include customers where `TotalSpent` is greater than 50,000.
* Order the results by `MostRecentOrder` in descending order.
* Limit the output to the `TOP 7` results.

In [ ]:
USE WideWorldImporters;
-- Using Common Table Expression (CTE)
WITH CustomerTotals AS (
    SELECT 
        o.CustomerID,
        SUM(ol.Quantity * ol.UnitPrice) AS TotalSpent,
        MAX(o.OrderDate) AS MostRecentOrder
    FROM Sales.Orders AS o
    JOIN Sales.OrderLines AS ol 
         ON o.OrderID = ol.OrderID
    GROUP BY o.CustomerID
)
SELECT TOP 7
       c.CustomerID,
       c.CustomerName,
       ct.TotalSpent,
       ct.MostRecentOrder
FROM CustomerTotals AS ct
JOIN Sales.Customers AS c
     ON c.CustomerID = ct.CustomerID
WHERE ct.TotalSpent > 50000
ORDER BY ct.MostRecentOrder DESC;
-- Filters customers who spent over $50k and shows their last order date.

### Proposition 10: Categorize products by price

**Functional Specification**
* Create a CTE named `ProductCategories`.
* Inside the CTE, select product information from the `Warehouse.StockItems` table.
* Filter the records to only include a specific list of `StockItemID`s (1, 3, 8, 177, 206).
* Use a `CASE` expression to create a new column `PriceCategory` based on the `UnitPrice`: 'Cheap' (< 10), 'Moderate' (10-100), or 'Expensive' (> 100).
* The final query selects all columns from the CTE.
* Order the results by `StockItemID`.

In [ ]:
USE WideWorldImporters;
-- Using Common Table Expression (CTE)
WITH ProductCategories AS
(
    SELECT 
        StockItemID,
        StockItemName,
        UnitPrice,
        CASE 
            WHEN UnitPrice < 10 THEN 'Cheap'
            WHEN UnitPrice BETWEEN 10 AND 100 THEN 'Moderate'
            ELSE 'Expensive'
        END AS PriceCategory
    FROM Warehouse.StockItems
    WHERE StockItemID IN (1, 3, 8, 177, 206)  -- pick specific products
)
SELECT 
    StockItemID,
    StockItemName,
    UnitPrice,
    PriceCategory
FROM ProductCategories
ORDER BY StockItemID;
-- Groups products as Cheap, Moderate, or Expensive using CASE.